# do a transitive query of a wikidata concept and search CU Experts elastic endpoint for all people who have this concept in their elastic document

### Use anaconda shell to add dependencies. eg: conda install -c conda-forge sparqlwrapper

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import namedtuple
import json
import requests

endpoint_url = "https://query.wikidata.org/sparql"
ES_person_endpoint_url = "https://experts.colorado.edu/es/fis/person/_search?size=5000"
ES_pub_endpoint_url = "https://experts.colorado.edu/es/fis/person/_search"
headers = {'Content-type': 'application/json'}
search_term = "artificial intelligence"
person_termlist = dict()

# Find the wikidata entity where the wikidata entity label matches the search_term string AND it's a concept - Q151885
# Then find all subclasses of that concept term in wikidata
query = """SELECT ?subitem ?subitemLabel
WHERE {

  ?item ?label \"""" + search_term + """\"@en.
    ?item wdt:P31* wd:Q11862829.
  ?subitem wdt:P279+ ?item.
  SERVICE wikibase:label { bd:serviceParam wikibase:language \"en\". }
  } """



In [2]:
def WdataSearch(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


In [3]:
def ESsearch(uri, term):
    """Simple Elasticsearch Query"""
    query = json.dumps({
        "query": {
#            "match": {
#                "researchArea.name.exact": term
#            }
             "query_string": { "query": term }
        }
    })
    response = requests.get(uri, headers=headers, data=query)
    results = json.loads(response.text)
    return results



In [4]:
print ("Only displaying results with a search score > .5")
print (search_term)
ESresults = ESsearch(ES_person_endpoint_url,search_term)
for eresult in ESresults['hits']['hits']:
      if eresult["_score"] >= 4:
        #person=personterms(search_term,eresult["_score"])
        #person.name = eresult["_source"]["name"]
        termlist=[]
        termscore={search_term,eresult["_score"]}
        termlist.append(termscore)
        person_termlist[eresult["_source"]["name"]]=termlist
        print (eresult["_score"], eresult["_source"]["name"], eresult["_source"]["uri"])

Wresults = WdataSearch(endpoint_url, query)

print (query)

for result in Wresults["results"]["bindings"]:
   print ("\n")
   print (result['subitemLabel']['value'], result['subitem']['value'])
   ESresults = ESsearch(ES_person_endpoint_url,result['subitemLabel']['value'])
   print ("Total people who match in CU Experts: ", ESresults['hits']['total'])
   for eresult in ESresults['hits']['hits']:
      if eresult["_score"] >= 4:
        print (eresult["_score"], eresult["_source"]["name"], eresult["_source"]["uri"])
        termscore={result['subitemLabel']['value'],eresult["_score"]}
        if eresult["_source"]["name"] in person_termlist:
            person_termlist[eresult["_source"]["name"]].append(termscore)
        else:
            termlist=[]
            termlist.append(termscore)
            person_termlist[eresult["_source"]["name"]]=termlist

print ("\nList of persons and thier terms")
for result in person_termlist:
    print(result,": ")
    for terms in person_termlist[result]:
       print("\t", terms)


Only displaying results with a search score > .5
artificial intelligence
13.635081 Heckman, Christoffer Roger https://experts.colorado.edu/individual/fisid_155294
13.070017 Kann, Katharina Lisa Maria https://experts.colorado.edu/individual/fisid_166417
11.935764 Hunter, Lawrence E https://experts.colorado.edu/individual/fisid_143568
11.631038 Laguna, Manuel https://experts.colorado.edu/individual/fisid_102975
11.605601 Monteleoni, Claire Elizabeth https://experts.colorado.edu/individual/fisid_163979
11.605601 Burke, Robin D. https://experts.colorado.edu/individual/fisid_165005
11.269469 Liu, Youjian https://experts.colorado.edu/individual/fisid_126283
11.241237 Roncone, Alessandro https://experts.colorado.edu/individual/fisid_164509
11.208887 Stoica, Maria https://experts.colorado.edu/individual/fisid_157419
11.120407 Gurari, Danna https://experts.colorado.edu/individual/fisid_167410
10.928836 Surden, Harry https://experts.colorado.edu/individual/fisid_146083
10.58853 O'Reilly, Randall